In [2]:
import requests
from bs4 import BeautifulSoup
import os
from time import time
import string
import sys
import pandas as pd 
import numpy as np
#!{sys.executable} -m pip install imdbpy
import imdb
import time 
# so I guess something I need to well.... CHECK 
# is is the list of those prep_titles the same as the dimensions of the dataframe? 
# or... well if we are going off of everythign tin the dataframe... we should end upw ith the same enght... yeah that's true... 
# I'm just now sure how we got less 

In [4]:
# from Pedro 
def get_data():

    """ Scrapes the data from https://www.the-numbers.com/movie/budgets/all """
    soup = BeautifulSoup(website_url, "lxml")
    movies = soup.find_all("tr")[1:]
    for movie in movies:
        original_title = movie.find("b").string
        # Gets the title that will match the titles we have saved as movie scripts
        processed_title = original_title.replace(": ", "-").replace(" ", "-").replace("â\x80\x99", "\'")
        if processed_title[:4] == "The-":
            processed_title = processed_title[processed_title.find("The-")+4:] + "," + "-The"
        # If the script is in our directory, gets the data
        if processed_title not in prep_title:  # Avoids duplicates
            org_title.append(original_title.replace("â\x80\x99", "\'"))
            prep_title.append(processed_title)
            release_date.append(movie.find("a").string)
            numbers = movie.find_all("td", {"class": "data"})
            #prod_budget.append(int(numbers[1].string[1:].replace(",", "")))
            #dom_gross.append(int(numbers[2].string[1:].replace(",", "")))
            #world_gross.append(int(numbers[3].string[1:].replace(",", "")))

In [5]:
# also from pedro
org_title, prep_title = [], []
release_date, prod_budget, dom_gross, world_gross = [], [], [], []
website_url = requests.get("https://www.the-numbers.com/movie/budgets/all").text
for i in range(101, 5800, 100):
    website_url = requests.get("https://www.the-numbers.com/movie/budgets/all/" + str(i)).text
    get_data()

In [6]:
# Sean City from here on out 
# Get the necessary columns of the movies (we search all, not just the successful ones in this run)
df = pd.read_csv('success_data.csv', index_col=0)
df = pd.DataFrame({"Processed Title": prep_title, "Release Date": release_date}, index=org_title)
df['Year'] = df.apply(lambda row: row['Release Date'].split(' ')[2], axis=1)

df.head()

,Processed Title,Release Date,Year
Monsters vs. Aliens,Monsters-vs.-Aliens,"Mar 27, 2009",2009
Cars 3,Cars-3,"Jun 16, 2017",2017
G.I. Joe: The Rise of Cobra,G.I.-Joe-The-Rise-of-Cobra,"Aug 7, 2009",2009
Wild Wild West,Wild-Wild-West,"Jun 30, 1999",1999
The Mummy: Tomb of the Dragon Emperor,"Mummy-Tomb-of-the-Dragon-Emperor,-The","Aug 1, 2008",2008


In [ ]:
df['imdb_objects'] = ['NO MOVIE FOUND!!!!'] * df.shape[0]
df['imdb_id'] = ['NO ID FOUND'] * df.shape[0]
# we essntially add columns to see if a movie or its id ws found
# I ended up having to delete the movie column bc it made the dataframe misbehave, hope to get that to work eventuallyu 
# BUT all you need is the ID to pull the summaries/synopses, so it's fine

In [21]:
# below I create the object that will create the connection with the IMDB api... you may need to referesh it if you time out 
# also, there is an example of how you can  search for a movie based on its title 

#NOTE you can only get a few details from a movie if you search by its title... to get the interesting stuff, 
# you need to grab it by its id, whichw e get in the search. 

# btw, everything is kinda stored in a dictionary like structure in the movie objects. 
len(imdb_)

5562

In [24]:
ia = imdb.IMDb()
fg = ia.search_movie('Forrest Gump')[0]

# check if budget data == original webtitle
# compare original title to new title 

# go to IMDB, compare the budget 
str(fg.data['year'])
fg.movieID

'0109830'

In [9]:
# FAILY CERTAIN i RAN THIS ONE MAYN3

test = 0
imdb_movieID2 = []
imdb_tits2 = []
moviez = []
full_matches = []
fm_id = []
full_movie = []
for index, data in df.iterrows():
    #print(index)
    # so for each title in our success data, we will enter a query to imdb's API
    #imdbS = ia.search_movie((data[0]+' ' + str(data[2])).replace('-',' '))
    imdbS = ia.search_movie((index+' ' + str(data[2])))

    #tit_words = data[0].replace('-',' ').split(' ')
    tit_words = index
    # for every candidate in our query...
    bk = False
    for candidate in imdbS:
        
        if bk == True:
            break
        # so even if there is just one movie retrieved.... it will just search 1 time 
        if 'year' in candidate.data:
            # check if the movie object has a year attribute. 
            #print(candidate.data)
            #now check if the year there matches the release year in our dataset
            if int(data[2]) == candidate.data['year']:
                if candidate.data['title'].lower() == index.lower():
                    imdb_tits2.append(candidate)
                    imdb_movieID2.append(candidate.movieID)
                    moviez.append(candidate)
                    full_matches.append(candidate)
                    fm_id.append(candidate.movieID)
                    full_movie.append(candidate)
                    bk = True
                    break
                else:
                    for word in tit_words:
                        if word.lower() in candidate.data['title'].lower():
                            imdb_tits2.append(candidate)
                            imdb_movieID2.append(candidate.movieID)
                            moviez.append(candidate)
                            full_matches.append('No Full Match')
                            fm_id.append(0)
                            full_movie.append('No Full Movie')
                            
                            bk = True
                            break
                # so once it finds one title word, it appends, but as constructed currently
                # it will keep looking for more titles... so once we append 
                # we want to tell the loop agove it to break... 
                
        else:
            imdb_tits2.append('No Matches Found!!!')
            imdb_movieID2.append(0)
            moviez.append('No Movie')
            full_movie.append('No Full Movie')
            full_matches.append('No Full Match')
            fm_id.append(0)
            bk = True
    else: 
        continue
            
        
print(imdb_tits, imdb_movieID)
# ok, so I think I need to rework this, man! 
# basically, here's what we're gonna do, MAN! 
# raise another mofuking error... 

[<Movie id:0892782[http] title:_Monsters vs. Aliens (2009)_>, <Movie id:3606752[http] title:_Cars 3 (2017)_>, <Movie id:1046173[http] title:_G.I. Joe: The Rise of Cobra (2009)_>, <Movie id:0120891[http] title:_Wild Wild West (1999)_>, <Movie id:0859163[http] title:_The Mummy: Tomb of the Dragon Emperor (2008)_>, <Movie id:0413099[http] title:_Evan Almighty (2007)_>, <Movie id:0114898[http] title:_Waterworld (1995)_>, <Movie id:1972591[http] title:_King Arthur: Legend of the Sword (2017)_>, <Movie id:1335975[http] title:_47 Ronin (2013)_>, 'No Matches Found!!!', 'No Matches Found!!!', 'No Matches Found!!!', <Movie id:4881806[http] title:_Jurassic World: Fallen Kingdom (2018)_>, <Movie id:2015381[http] title:_Guardians of the Galaxy (2014)_>, <Movie id:1228705[http] title:_Iron Man 2 (2010)_>, <Movie id:1843866[http] title:_Captain America: The Winter Soldier (2014)_>, <Movie id:2103281[http] title:_Dawn of the Planet of the Apes (2014)_>, <Movie id:0338348[http] title:_The Polar Express

In [22]:
bad = [b1 for b1 in imdb_tits if b1 == 'No Matches Found!!!']
#squares = [n**2 for n in numbers]
len(imdb_tits) - len(bad)
# ok, so there were 945 movies that we couldn't find the title for, or at least, a title that was released in the same year 
# as the movie we have success data for 
len(imdb_movieID)
#== df.shape[0]

6181

In [189]:
mini_df = df.iloc[1:10,:]
mini_df

['No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Found!!!',
 'No Matches Fou

In [139]:
#for title in titles:
#    if title[:4] == "The-":
#        titles_better.append(title[title.find("The-")+4:] + "," + "-The")
#    else:
#        titles_better.append(title)
# we arleady have the formatte 
# Tries to get the scripts for all the movie titles (most will fail)
miniboi = towgz.head()

new_title = []
for title in towgz[towgz.columns[0]]:
    title = title.replace('?','')
    new_title.append(title.replace('/',''))
    
    
    
new_title = pd.Series(new_title)
towgz['new_title'] = new_title


towgz['has_synopsis'] = ['NO'] * towgz.shape[0]
towgz['has_plot'] = ['NO'] * towgz.shape[0]
#mini_df = df['imdb_id'] == 'NO ID FOUND' 
for index,row in towgz.iterrows():
    if row[3] == "NO ID FOUND":
        continue
    movie = ia.get_movie(row[3])
    plot = movie.get('plot')
    synopsis = movie.get('synopsis')
    # case where we ONLY have a synopsis and no plot summary
    if (plot == None) & (synopsis != None):
        with open(os.getcwd() + "\\synopses\\" + row[6] + "_synopsis.txt", "w") as sy:
            sy.write(synopsis[0])
            row[4] = 'YES'
    elif(plot != None) & (synopsis == None):
        with open(os.getcwd() + "\\summaries\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
            row[5] = 'YES'
                # the case where we have BOTH as acceptable files!!! 
    elif(plot != None) & (synopsis !=None):
        with open(os.getcwd() + "\\summaries\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
        with open(os.getcwd() + "\\synopses\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
            row[4] = 'YES'
            row[5] = 'YES'
        
    
            
    #print('synopsis')
    #print(type(synopsis))
    #print(synopsis[0])
#end = time() - start
#print("The scraping process took", round(end), "seconds")  # The scraping process took 2268 seconds

# Gets the titles whose scripts we actually managed to get
#ready_scripts = [file_name[:file_name.find("_")] for file_name
#                 in os.listdir(os.getcwd() + "/scripts/")
#                 if ".txt" in file_name]
#print("Out of", len(titles), "movies,", "we only got", len(ready_scripts), "scripts")  # Out of 5714 movies, we only got 703 scripts

TypeError: must be str, not float

In [111]:
towgz.head()

Monsters-vs.-Aliens
Cars-3
G.I.-Joe-The-Rise-of-Cobra
Wild-Wild-West
Mummy-Tomb-of-the-Dragon-Emperor,-The
Evan-Almighty
Waterworld
King-Arthur-Legend-of-the-Sword
47-Ronin
Singularity
Jurassic-World-Fallen-Kingdom
Guardians-of-the-Galaxy
Iron-Man-2
Captain-America-The-Winter-Soldier
Dawn-of-the-Planet-of-the-Apes
Polar-Express,-The
Snow-White-and-the-Huntsman
Terminator-3-Rise-of-the-Machines
Van-Helsing
Tomorrowland
Alice-Through-the-Looking-Glass
Shrek-Forever-After
Doctor-Strange
Big-Hero-6
How-to-Train-Your-Dragon
Wreck-It-Ralph
Interstellar
Independence-Day-Resurgence
Cowboys-and-Aliens
Beauty-and-the-Beast
Aquaman
Shrek-the-Third
Inception
Hunger-Games-Mockingjay---Part-2,-The
Fast-and-Furious-6
Godzilla
X-Men-First-Class
Curious-Case-of-Benjamin-Button,-The
Sorcerer's-Apprentice,-The
Poseidon
Warcraft
Deepwater-Horizon
Chronicles-of-Narnia-The-Voyage-of-the-Dawn-Treader,-The
Terminator-Genisys
Pacific-Rim-Uprising
Alexander
Captain-Marvel
War-for-the-Planet-of-the-Apes
Pearl-Ha

Cape-Fear
English-Patient,-The
Hotel-for-Dogs
Guess-Who
This-is-40
L.A.-Confidential
Sky-High
In-&-Out
A-Nightmare-on-Elm-Street
Cell,-The
Species
Benchwarmers,-The
Secretariat
Man-in-the-Iron-Mask,-The
Friends-with-Benefits
Neighbors-2-Sorority-Rising
TMNT
Radio
Sicario-Day-of-the-Soldado
Old-Dogs
Malcolm-X
Poltergeist
Goosebumps-2-Haunted-Halloween
12-Strong
Underworld-3-Rise-of-the-Lycans
Must-Love-Dogs
License-to-Wed
Donnie-Brasco
Max-Payne
Resident-Evil
Ladykillers,-The
Something-Borrowed
Nativity-Story,-The
In-Time
Back-up-Plan,-The
J.-Edgar
Bounce
Mile-22
Hot-Pursuit
Concussion
Foreigner,-The
Closer
Street-Fighter
Black-Knight
Bad-News-Bears,-The
Pianist,-The
House-of-Wax
From-Hell
Adrift
Mirrors
Underworld-Blood-Wars
Queen-of-the-Damned
Nanny-McPhee-and-the-Big-Bang
Untraceable
Predator-2
Abduction
Flash-Gordon
Superhero-Movie
Blast-from-the-Past
Alex-Cross
Jersey-Girl
Midnight-in-the-Garden-of-Good-and-Evil
Bridget-Jones's-Baby
Heist
Hoffa
Whiskey-Tango-Foxtrot
Ella-Enchanted


Connie-&-Carla
Machete-Kills
Sweet-Charity
Doogal
Proof
An-American-Carol
Willard
Strange-Wilderness
Little-Boy
Topsy-Turvy
Chasing-Mavericks
A-Most-Violent-Year
A-Dangerous-Method
A-Scanner-Darkly
Bandslam
Alone-in-the-Dark
Birth
Hands-of-Stone
Passchendaele
Flash-of-Genius
I'm-Not-There
Cold-Light-of-Day,-The
Brothers-Bloom,-The
Synecdoche,-New-York
Suspiria
LBJ
Mononoke-hime
Bon-Voyage
My-All-American
Can't-Stop-the-Music
Proposition,-The
Marci-X
Equilibrium
Dylan-Dog-Dead-of-Night
Oogieloves-in-the-BIG-Balloon-Adventure,-The
Children-of-Huang-Shi,-The
Yards,-The
Middle-Men
JÃ¬yÃ¬-dÃ shÄ«
All-Good-Things
By-the-Sea
Steamboy
Game-of-Their-Lives,-The
Rapa-Nui
Tempest,-The
é·æ±ä¸è-(CJ7)
Burning-Plain,-The
Les-Couloirs-du-temps-Les-visiteurs-2
People-I-Know
Regression
Admiral
Zambezia
Three-Kingdoms-Resurrection-of-the-Dragon
Touch,-The
Survivor
History-of-Love,-The
Butterfly-on-a-Wheel
Dwegons-and-Leprechauns
Fight-Valley
Thousand-Miles,-The
Bird-Box
Der-Baader-Meinhof-Komplex
Sha

Sphinx
Iceman,-The
Free-Fire
A-Better-Life
Spider
Gun-Shy
Nicholas-Nickleby
Joneses,-The
Krrish
Killer-Joe
Churchill
Cecil-B.-Demented
50-to-1
Owning-Mahowny
Brothers-Solomon,-The
My-Blueberry-Nights
Illuminata
Coriolanus
Parkland
Swept-Away
War,-Inc.
Shaolin-Soccer
Brown-Bunny,-The
King-of-California
Rien-ne-va-plus
La-femme-de-chambre-du-Titanic
Imaginary-Heroes
High-Heels-and-Low-Lifes
World's-Greatest-Dad
Cinco-de-Mayo,-La-Batalla
Welcome-to-the-Rileys
Kicks
Free-Style
High-School
Severance
Edmond
Joheunnom-nabbeunnom-isanghannom
Police-Academy-7-Mission-to-Moscow
Blood-Done-Sign-My-Name
Good-Guy,-The
Motherhood
Eulogy
Elsa-&-Fred
An-Alan-Smithee-Film-Burn-Hollywood-Burn
Open-Road,-The
Strangerland
Hross-i-oss
Janky-Promoters
Blonde-Ambition
Oxford-Murders,-The
Barbecue
Reef,-The
White-Noise-2-The-Light
American-Heist
It's-a-Wonderful-Afterlife
You-Got-Served-Beat-The-World
Fifty-Dead-Men-Walking
Plastic
Jungle-Shuffle
Adam-Resurrected
Devil's-Tomb,-The
Good-Intentions
Partition
De

Phat-Girlz
Woman-Thou-Art-Loosed
Real-Women-Have-Curves
Gallipoli
Water
Don't-Think-Twice
Swiss-Army-Man
East-is-East
Whipped
Kama-Sutra
Frances-Ha
Please-Give
Willy-Wonka-&-the-Chocolate-Factory
Warlock-The-Armageddon
8-Heads-in-a-Duffel-Bag
Days-of-Heaven
Thirteen-Conversations-About-One-Thing
Compadres
Jawbreaker
Basquiat
Tsotsi
Letters-to-God
Happiness
DysFunkTional-Family
Tusk
Wall,-The
Elephant
Damsels-in-Distress
Oldboy
Hobo-with-a-Shotgun
D-Train,-The
Bachelorette
Fish-Tank
Everybody-Wants-to-Be-Italian
Creature
Bounce-Back,-The
Freeway
Love's-Abiding-Joy
Dead-Alive
Chain-Letter
Tim-and-Eric's-Billion-Dollar-Movie
Holly
Eclipse,-The
Grand,-The
Sommersturm
Fort-McCoy
Just-Looking
Before-We-Go
Divide,-The
Tanner-Hall
My-Big-Fat-Independent-Movie
Demonic
Ã -l\'intÃ©rieur
They-Came-Together
Open-Road
Alice-in-Wonderland
Barry-Munday
Deported,-The
Chicago-Overcoat
Never-Back-Down-2-The-Beatdown
AWOL-72
30-Nights-Of-Paranormal-Activity-With-The-Devil-Inside-The-Girl-With-The-Dragon-T

Krisha
Mutual-Appreciation
Funny-Ha-Ha
Down-Terrace
Paraphobia
Clerks
Pink-Narcissus
Emily
Deep-Throat
In-the-Company-of-Men
Terrorist,-The
Exeter
Manito
Ten
Dutch-Kills
Slacker
Dry-Spell
Flywheel
All-Superheroes-Must-Die
Front-Man,-The
Steel-Spirit
Ridges,-The
Puffy-Chair,-The
Breaking-Upwards
Stories-of-Our-Lives
Pink-Flamingos
Grip-A-Criminal's-Story
Tin-Can-Man
Dayereh
Clean
Cure
On-the-Downlow
Bang
Rise-and-Fall-of-Miss-Thang,-The
Family-Motocross
Newlyweds
El-Mariachi
Primer
Cavite
Mongol-King,-The
Red-11
Following
Return-to-the-Land-of-Wonders
A-Plague-So-Pleasant
My-Date-With-Drew


In [142]:
#for title in titles:
#    if title[:4] == "The-":
#        titles_better.append(title[title.find("The-")+4:] + "," + "-The")
#    else:
#        titles_better.append(title)
# we arleady have the formatte 
# Tries to get the scripts for all the movie titles (most will fail)
miniboi = towgz.head()

new_title = []
for title in towgz[towgz.columns[0]]:
    tit = title.replace('/','')
    tit2 = tit.replace('*','')

    new_title.append(tit2.replace('?',''))
    #print(new_title)
#new_title = pd.Series(new_title)
new_title
towgz['new_title'] = pd.Series(new_title,index = towgz.index)


towgz['has_synopsis'] = ['NO'] * towgz.shape[0]
towgz['has_plot'] = ['NO'] * towgz.shape[0]
#mini_df = df['imdb_id'] == 'NO ID FOUND' 
for index,row in towgz.iterrows():
    if row[3] == "NO ID FOUND":
        continue
    movie = ia.get_movie(row[3])
    plot = movie.get('plot')
    synopsis = movie.get('synopsis')
    # case where we ONLY have a synopsis and no plot summary
    if (plot == None) & (synopsis != None):
        with open(os.getcwd() + "\\synopses\\" + row[6] + "_synopsis.txt", "w") as sy:
            sy.write(synopsis[0])
            row[4] = 'YES'
    elif(plot != None) & (synopsis == None):
        with open(os.getcwd() + "\\summaries\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
            row[5] = 'YES'
                # the case where we have BOTH as acceptable files!!! 
    elif(plot != None) & (synopsis !=None):
        with open(os.getcwd() + "\\summaries\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
        with open(os.getcwd() + "\\synopses\\" + row[6] + "_summary.txt", "w") as su:
            su.write(plot[0])
            row[4] = 'YES'
            row[5] = 'YES'
        
    
            
    #print('synopsis')
    #print(type(synopsis))
    #print(synopsis[0])
#end = time() - start
#print("The scraping process took", round(end), "seconds")  # The scraping process took 2268 seconds

# Gets the titles whose scripts we actually managed to get
#ready_scripts = [file_name[:file_name.find("_")] for file_name
#                 in os.listdir(os.getcwd() + "/scripts/")
#                 if ".txt" in file_name]
#print("Out of", len(titles), "movies,", "we only got", len(ready_scripts), "scripts")  # Out of 5714 movies, we only got 703 scripts

In [25]:

for index, data in df.iterrows():
    #print(index)
    # so for each title in our success data, we will enter a query to imdb's API
    #imdbS = ia.search_movie((data[0]+' ' + str(data[2])).replace('-',' '))
    imdbS = ia.search_movie((index+' ' + str(data[2])))

    #tit_words = data[0].replace('-',' ').split(' ')
    tit_words = index
    # for every candidate in our query...
    bk = False
    for candidate in imdbS:
        
        if bk == True:
            break
        # so even if there is just one movie retrieved.... it will just search 1 time 
        if 'year' in candidate.data:
            # check if the movie object has a year attribute. 
            #print(candidate.data)
            #now check if the year there matches the release year in our dataset
            if int(data[2]) == candidate.data['year']:
                if candidate.data['title'].lower() == index.lower():
                    df.loc[index,'imdb_objects'] = candidate
                    df.loc[index,'imdb_id'] = candidate.movieID
                    #moviez.append(candidate)
                    #full_matches.append(candidate)
                    #fm_id.append(candidate.movieID)
                    #full_movie.append(candidate)
                    #bk = True
                    #break
                else:
                    for word in tit_words:
                        if word.lower() in candidate.data['title'].lower():
                            df.loc[index,'imdb_objects'] = candidate
                            df.loc[index,'imdb_id'] = candidate.movieID
                           # full_matches.append('No Full Match')
                           # fm_id.append(0)
                            #full_movie.append('No Full Movie')
                            
                            bk = True
                            break
                # so once it finds one title word, it appends, but as constructed currently
                # it will keep looking for more titles... so once we append 
                # we want to tell the loop agove it to break... 
                
        else:
            imdb_tits.append('No Matches Found!!!')
            imdb_movieID.append(0)
            moviez.append('No Movie')
            full_movie.append('No Full Movie')
            full_matches.append('No Full Match')
            fm_id.append(0)
            bk = True 
            
    else: 
        continue
            
        
print(imdb_tits, imdb_movieID)
# ok, so I think I need to rework this, man! 
# basically, here's what we're gonna do, MAN! 
# raise another mofuking error... 

[<Movie id:0892782[http] title:_Monsters vs. Aliens (2009)_>, <Movie id:3606752[http] title:_Cars 3 (2017)_>, <Movie id:1046173[http] title:_G.I. Joe: The Rise of Cobra (2009)_>, <Movie id:0120891[http] title:_Wild Wild West (1999)_>, <Movie id:0859163[http] title:_The Mummy: Tomb of the Dragon Emperor (2008)_>, <Movie id:0413099[http] title:_Evan Almighty (2007)_>, <Movie id:0114898[http] title:_Waterworld (1995)_>, <Movie id:1972591[http] title:_King Arthur: Legend of the Sword (2017)_>, <Movie id:1335975[http] title:_47 Ronin (2013)_>, 'No Matches Found!!!', 'No Matches Found!!!', 'No Matches Found!!!', <Movie id:4881806[http] title:_Jurassic World: Fallen Kingdom (2018)_>, <Movie id:2015381[http] title:_Guardians of the Galaxy (2014)_>, <Movie id:1228705[http] title:_Iron Man 2 (2010)_>, <Movie id:1843866[http] title:_Captain America: The Winter Soldier (2014)_>, <Movie id:2103281[http] title:_Dawn of the Planet of the Apes (2014)_>, <Movie id:0338348[http] title:_The Polar Express

In [129]:

new_title = []
for title in towgz[towgz.columns[0]]:
    new_title.append(title.replace('?',''))
    #print(new_title)
#new_title = pd.Series(new_title)
new_title
towgz['new_title'] = pd.Series(new_title,index = towgz.index)
towgz['new_title']


Monsters vs. Aliens                                        Monsters-vs.-Aliens
Cars 3                                                                  Cars-3
G.I. Joe: The Rise of Cobra                         G.I.-Joe-The-Rise-of-Cobra
Wild Wild West                                                  Wild-Wild-West
The Mummy: Tomb of the Dragon Emperor    Mummy-Tomb-of-the-Dragon-Emperor,-The
Evan Almighty                                                    Evan-Almighty
Waterworld                                                          Waterworld
King Arthur: Legend of the Sword               King-Arthur-Legend-of-the-Sword
47 Ronin                                                              47-Ronin
Singularity                                                        Singularity
Jurassic World: Fallen Kingdom                   Jurassic-World-Fallen-Kingdom
Guardians of the Galaxy                                Guardians-of-the-Galaxy
Iron Man 2                                          

In [28]:
df.shape[0]

5562

In [65]:
towgz = df.drop('imdb_objects',axis=1)

In [148]:
towgz[towgz.iloc[:,5]=='YES'].shape[0]

2634

In [150]:
towgz.tail(15)

,Processed Title,Release Date,Year,imdb_id,has_synopsis,has_plot,new_title
Cure,Cure,"Jul 6, 2001",2001,0295942,NO,NO,Cure
On the Downlow,On-the-Downlow,"May 28, 2004",2004,0472070,NO,NO,On-the-Downlow
Bang,Bang,"Apr 1, 1996",1996,0305002,NO,NO,Bang
The Rise and Fall of Miss Thang,"Rise-and-Fall-of-Miss-Thang,-The","Aug 14, 2008",2008,NO ID FOUND,NO,NO,"Rise-and-Fall-of-Miss-Thang,-The"
Family Motocross,Family-Motocross,"May 19, 2015",2015,NO ID FOUND,NO,NO,Family-Motocross
Newlyweds,Newlyweds,"Jan 13, 2012",2012,2198934,NO,NO,Newlyweds
El Mariachi,El-Mariachi,"Feb 26, 1993",1993,6112998,NO,YES,El-Mariachi
Primer,Primer,"Oct 8, 2004",2004,0523814,NO,NO,Primer
Cavite,Cavite,"May 26, 2006",2006,NO ID FOUND,NO,NO,Cavite
The Mongol King,"Mongol-King,-The","Dec 31, 2004",2004,NO ID FOUND,NO,NO,"Mongol-King,-The"


In [70]:
import os
import 

for i in df.index[0:10]:
    print(i + '___' +str(df.loc[i,['Year']].values[0]) )


Avatar___2009
Pirates of the Caribbean: On Stranger Tides___2011
Avengers: Age of Ultron___2015
Star Wars Ep. VIII: The Last Jedi___2017
Star Wars Ep. VII: The Force Awakens___2015
Avengers: Infinity War___2018
Pirates of the Caribbean: At World's End___2007
Justice League___2017
Spectre___2015
The Dark Knight Rises___2012


In [15]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np





In [133]:
'hello'.translate(None,['\\'])

TypeError: translate() takes exactly one argument (2 given)

In [155]:
sucdat = pd.read_csv('success_data.csv')

In [166]:
#sucdat.index = sucdat['Unnamed: 0']
#sucdat.drop('Unnamed: 0',axis=1)
sucdat.head()

,Unnamed: 0,Processed Title,Release Date,Production Budget ($),Domestic Gross ($),Worldwide Gross ($),Domestic ROI (%),Worldwide ROI (%)
Unnamed: 0,,,,,,,,
The Dark Knight Rises,The Dark Knight Rises,"Dark-Knight-Rises,-The","Jul 20, 2012",275000000,448139099.0,1.084439e+09,62.959672,294.341491
The Lone Ranger,The Lone Ranger,"Lone-Ranger,-The","Jul 2, 2013",275000000,89302115.0,2.600021e+08,-67.526504,-5.453776
The Hobbit: An Unexpected Journey,The Hobbit: An Unexpected Journey,"Hobbit-An-Unexpected-Journey,-The","Dec 14, 2012",250000000,303003568.0,1.017004e+09,21.201427,306.801427
The Hobbit: The Desolation of Smaug,The Hobbit: The Desolation of Smaug,"Hobbit-The-Desolation-of-Smaug,-The","Dec 13, 2013",250000000,258366855.0,9.603669e+08,3.346742,284.146742
The Hobbit: The Battle of the Five Armies,The Hobbit: The Battle of the Five Armies,"Hobbit-The-Battle-of-the-Five-Armies,-The","Dec 17, 2014",250000000,255119788.0,9.455776e+08,2.047915,278.231048


In [167]:
test = towgz.join(sucdat,how='inner',rsuffix="_pedro",lsuffix='_sean')

In [169]:
test.shape[0]

972

In [170]:
test

,Processed Title_sean,Release Date_sean,Year,imdb_id,has_synopsis,has_plot,new_title,Unnamed: 0,Processed Title_pedro,Release Date_pedro,Production Budget ($),Domestic Gross ($),Worldwide Gross ($),Domestic ROI (%),Worldwide ROI (%)
The 13th Warrior,"13th-Warrior,-The","Aug 27, 1999",1999,0120657,YES,YES,"13th-Warrior,-The",The 13th Warrior,"13th-Warrior,-The","Aug 27, 1999",125000000,32698899.0,61698899.0,-73.840881,-50.640881
The 15:17 to Paris,"15:17-to-Paris,-The","Feb 9, 2018",2018,7947250,NO,YES,"15:17-to-Paris,-The",The 15:17 to Paris,"15:17-to-Paris,-The","Feb 9, 2018",30000000,36276286.0,56096200.0,20.920953,86.987333
The 33,"33,-The","Nov 13, 2015",2015,5543344,NO,NO,"33,-The",The 33,"33,-The","Nov 13, 2015",25000000,12227722.0,28400715.0,-51.089112,13.602860
The 40 Year-old Virgin,"40-Year-old-Virgin,-The","Aug 19, 2005",2005,0405422,YES,YES,"40-Year-old-Virgin,-The",The 40 Year-old Virgin,"40-Year-old-Virgin,-The","Aug 19, 2005",26000000,109449237.0,177344230.0,320.958604,582.093192
The 5th Quarter,"5th-Quarter,-The","Mar 25, 2011",2011,NO ID FOUND,NO,NO,"5th-Quarter,-The",The 5th Quarter,"5th-Quarter,-The","Mar 25, 2011",6000000,408159.0,408159.0,-93.197350,-93.197350
The 5th Wave,"5th-Wave,-The","Jan 22, 2016",2016,5783654,NO,NO,"5th-Wave,-The",The 5th Wave,"5th-Wave,-The","Jan 22, 2016",38000000,34912982.0,111336398.0,-8.123732,192.990521
The 6th Day,"6th-Day,-The","Nov 17, 2000",2000,3309504,NO,NO,"6th-Day,-The",The 6th Day,"6th-Day,-The","Nov 17, 2000",82000000,34543701.0,96024898.0,-57.873535,17.103534
The A-Team,"A-Team,-The","Jun 11, 2010",2010,1386588,YES,YES,"A-Team,-The",The A-Team,"A-Team,-The","Jun 11, 2010",110000000,77222099.0,177241171.0,-29.798092,61.128337
The Abyss,"Abyss,-The","Aug 9, 1989",1989,0096785,NO,YES,"Abyss,-The",The Abyss,"Abyss,-The","Aug 9, 1989",70000000,54243125.0,54243125.0,-22.509821,-22.509821
The Accountant,"Accountant,-The","Oct 14, 2016",2016,3116528,NO,NO,"Accountant,-The",The Accountant,"Accountant,-The","Oct 14, 2016",40000000,86260045.0,152962083.0,115.650113,282.405208


In [173]:
towgz.to_csv('plot_frame.csv')

In [1]:
import pandas as pd

In [2]:
plot_frame = pd.read_csv('plot_frame.csv')

In [6]:
plot_frame[plot_frame['has_plot'] == 'YES']

,Unnamed: 0,Processed Title,Release Date,Year,imdb_id,has_synopsis,has_plot,new_title
0,Monsters vs. Aliens,Monsters-vs.-Aliens,"Mar 27, 2009",2009,1482967,YES,YES,Monsters-vs.-Aliens
1,Cars 3,Cars-3,"Jun 16, 2017",2017,7084926,NO,YES,Cars-3
10,Jurassic World: Fallen Kingdom,Jurassic-World-Fallen-Kingdom,"Jun 22, 2018",2018,9455824,NO,YES,Jurassic-World-Fallen-Kingdom
14,Dawn of the Planet of the Apes,Dawn-of-the-Planet-of-the-Apes,"Jul 11, 2014",2014,5244628,NO,YES,Dawn-of-the-Planet-of-the-Apes
16,Snow White and the Huntsman,Snow-White-and-the-Huntsman,"Jun 1, 2012",2012,2572402,NO,YES,Snow-White-and-the-Huntsman
18,Van Helsing,Van-Helsing,"May 7, 2004",2004,0406310,NO,YES,Van-Helsing
19,Tomorrowland,Tomorrowland,"May 22, 2015",2015,3098856,NO,YES,Tomorrowland
21,Shrek Forever After,Shrek-Forever-After,"May 21, 2010",2010,1640561,NO,YES,Shrek-Forever-After
26,Interstellar,Interstellar,"Nov 5, 2014",2014,4288316,NO,YES,Interstellar
28,Cowboys and Aliens,Cowboys-and-Aliens,"Jul 29, 2011",2011,0409847,YES,YES,Cowboys-and-Aliens
